In [5]:
import os

folder_path = '../face_data'
subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

for subfolder in subfolders:
    subfolder_path = os.path.join(folder_path, subfolder)
    file_count = len([f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))])
    print(f"{subfolder}: {file_count} files")

laugh: 511 files
none: 338 files
serious: 509 files
surprise: 539 files
ugly: 403 files
yawn: 292 files


In [6]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# 1. 하이퍼파라미터
IMG_SIZE = 64
DATA_PATH = '../face_data'
LABELS = ['laugh', 'serious', 'surprise', 'ugly', 'yawn']

# 2. 데이터 불러오기
X, y = [], []
for idx, label in enumerate(LABELS):
    folder_path = os.path.join(DATA_PATH, label)
    for file in os.listdir(folder_path):
        if file.endswith('.jpg'):
            img_path = os.path.join(folder_path, file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # 흑백
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            X.append(img)
            y.append(idx)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0  # 정규화
y = to_categorical(y)

# 3. 학습/검증 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 4. 모델 구성
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(LABELS), activation='softmax')
])

# 5. 컴파일 & 학습
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val))

# 6. 저장
model.save('expression_cnn_model.h5')


Epoch 1/30


c:\Users\SSAFY\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.3283 - loss: 1.5091 - val_accuracy: 0.6984 - val_loss: 0.9309
Epoch 2/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7260 - loss: 0.7759 - val_accuracy: 0.8137 - val_loss: 0.5115
Epoch 3/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8523 - loss: 0.4561 - val_accuracy: 0.8803 - val_loss: 0.3714
Epoch 4/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8974 - loss: 0.3017 - val_accuracy: 0.9091 - val_loss: 0.2785
Epoch 5/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9514 - loss: 0.1791 - val_accuracy: 0.9113 - val_loss: 0.2509
Epoch 6/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9520 - loss: 0.1528 - val_accuracy: 0.9424 - val_loss: 0.1959
Epoch 7/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9767 - loss: 0.0897 - val_accuracy: 0.9446 - val_loss: 0.1932
Epoch 8/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9761 - loss: 0.0913 - val_accuracy: 0.9446 - val_loss: 0.

In [7]:
import tensorflow as tf

# 저장된 모델 불러오기
model = tf.keras.models.load_model('expression_cnn_model.h5')

# 변환기 생성
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 저장
with open('expression_model.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\SSAFY\AppData\Local\Temp\tmpe8xtr57p\assets


INFO:tensorflow:Assets written to: C:\Users\SSAFY\AppData\Local\Temp\tmpe8xtr57p\assets


Saved artifact at 'C:\Users\SSAFY\AppData\Local\Temp\tmpe8xtr57p'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  2256124633232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2256124632352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2256125405216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2256125407504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2256125454720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2256125454192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2256125459472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2256120727504: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [8]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp

LABELS = ['laugh', 'serious', 'surprise', 'ugly', 'yawn']
IMG_SIZE = 64

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="expression_model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Face detection 초기화
mp_face = mp.solutions.face_detection
face_detection = mp_face.FaceDetection(model_selection=0, min_detection_confidence=0.7)

cap = cv2.VideoCapture(0)
print("🎥 웹캠 실행 중... 얼굴 감지하여 표정 분류")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 얼굴 감지
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detection.process(rgb)

    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            h, w, _ = frame.shape
            x, y, bw, bh = int(bboxC.xmin * w), int(bboxC.ymin * h), int(bboxC.width * w), int(bboxC.height * h)

            # 얼굴 crop & resize
            face = frame[y:y+bh, x:x+bw]
            if face.size == 0:  # 잘라낸 이미지가 비어있는지 확인
                continue        # 비어있으면 다음 프레임으로 넘어감
            gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            # 얼굴 이미지를 IMG_SIZE로 resize
            resized = cv2.resize(gray, (IMG_SIZE, IMG_SIZE))
            input_data = resized.reshape(1, IMG_SIZE, IMG_SIZE, 1).astype(np.float32) / 255.0

            # TFLite 추론
            interpreter.set_tensor(input_details[0]['index'], input_data)
            interpreter.invoke()
            output = interpreter.get_tensor(output_details[0]['index'])
            pred = np.argmax(output)
            conf = np.max(output)

            # 결과 출력
            label = LABELS[pred]
            cv2.putText(frame, f"{label} ({conf:.2f})", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x + bw, y + bh), (0, 255, 0), 2)
    else:
        cv2.putText(frame, "No face detected", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Expression Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\SSAFY\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


🎥 웹캠 실행 중... 얼굴 감지하여 표정 분류


c:\Users\SSAFY\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# 라벨
LABELS = ['laugh', 'serious', 'surprise', 'ugly', 'yawn']
IMG_SIZE = 64

# ✅ Keras 모델 로드 (.h5)
model = tf.keras.models.load_model('expression_cnn_model.h5')

# 웹캠 실행
cap = cv2.VideoCapture(0)
print("🎥 웹캠 실행 중... 'q'를 누르면 종료")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 프레임 → 흑백 → resize → 정규화
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (IMG_SIZE, IMG_SIZE))
    input_data = resized.reshape(1, IMG_SIZE, IMG_SIZE, 1).astype(np.float32) / 255.0

    # CNN 모델 추론
    pred_probs = model.predict(input_data, verbose=0)
    pred = np.argmax(pred_probs)
    conf = np.max(pred_probs)
    label = LABELS[pred]

    # 결과 화면에 표시
    cv2.putText(frame, f"{label} ({conf:.2f})", (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)

    cv2.imshow("Expression Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


🎥 웹캠 실행 중... 'q'를 누르면 종료


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(1, 64, 64, 1), dtype=float32). Expected shape (None, 936), but input has incompatible shape (1, 64, 64, 1)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 64, 64, 1), dtype=float32)
  • training=False
  • mask=None
  • kwargs=<class 'inspect._empty'>